#1. importar bibliotecas

In [ ]:
!pip install pandas
!pip install yfinance
!pip install streamlit

import pandas as pd
import yfinance as yf
import datetime as dt
import streamlit as st

In [4]:
st.set_page_config( page_title="INDICADORES FINANCEIROS", layout="wide")

In [5]:
st.title("INDICADORES DA BOLSA")
st.caption("Esse Dashboard foi gerado com enfoque para monitorar as ações BEST e algumas FIIs. Empresas de Bancos, Energia, Sustentabilidade e "
             "Telecomunicação")


2023-08-22 15:06:29.516 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

# 2. Chamar a API

In [6]:
end_date = dt.datetime.today()
start_date =  dt.datetime(end_date.year-1,end_date.month,end_date.day)
lista_empresa = ["VALE3.SA", "ITSA4.SA", "PETR4.SA", "BBAS3.SA", "IRBR3.SA", "KLBN11.SA", "KNRI11.SA", "HGLG11.SA",
                 "SUZB3.SA", "UNIP6.SA", "TASA4.SA", "TRPL4.SA", "TAEE11.SA", "CMIG4.SA"]

In [7]:
with st.container():
    st.header("Insira as informações solicitadas")

    col1, col2, col3 = st.columns(3)

    with col1:
        ativo = st.selectbox("Selecione os ativo desejado:", options=lista_empresa)
    with col2:
        data_inicial = st.date_input("Selecione a Data inicial", start_date)
    with col3:
        data_final = st.date_input("Selecione a Data final", end_date)


#3. Gera o df

In [8]:
df = yf.download(tickers=ativo , start = data_inicial, end = data_final, actions=True)

[*********************100%%**********************]  1 of 1 completed


#4. Criando metricas

In [9]:
ult_atualizacao = df.index.max() #Data da ultima att
ult_cotacao = round(df.loc[df.index.max(), "Adj Close"], 2) #Pega a última cotacao
menor_cotacao = round(df["Adj Close"].min(), 2) #Pega a menor cotacao do periodo
maior_cotacao = round(df["Adj Close"].max(), 2) #Pega a maior cotacao do periodo
prim_cotacao = round(df.loc[df.index.min(), "Adj Close"],2 ) #Pega a primeira cotacao encontrada
delta = round(((ult_cotacao- prim_cotacao)/ prim_cotacao)*100,2) #A variacao da cotacao no periodo
data = df[df["Dividends"]>0]
dividendo = data["Dividends"].mean()
dividendo_soma = data["Dividends"].sum()
historico = data["Dividends"].count()
Pl = ult_cotacao / dividendo_soma
dividen_max = data["Dividends"].max()
dividen_min = data["Dividends"].min()
dividend_yeld = round(dividendo_soma/ult_cotacao,4)*100

In [10]:
print(data)

                 Open       High        Low      Close  Adj Close    Volume  \
Date                                                                          
2022-12-13  86.250000  87.400002  85.629997  86.160004  81.839508  31559400   
2023-03-14  83.120003  84.349998  82.900002  83.980003  81.520630  18077900   
2023-08-14  62.980000  63.040001  61.900002  61.900002  61.900002  41056900   

            Dividends  Stock Splits  
Date                                 
2022-12-13   0.292013           0.0  
2023-03-14   1.827646           0.0  
2023-08-14   1.918472           0.0  


In [11]:
with st.container():

    st.subheader(f"Indicadores principais da ação {ativo}")
    col11, col12, col13, col14, col15 = st.columns(5)

    with col11:
        st.metric(f"Última atualização - {ult_atualizacao} "," R$ {:,.2f}".format(ult_cotacao),f"{delta}%" )

    with col12:
        st.metric(f"Menor cotação do período: "," R$ {:,.2f}".format(menor_cotacao))

    with col13:
        st.metric(f"Maior cotação do período: "," R$ {:,.2f}".format(maior_cotacao))

    with col14:
        st.metric(f"Preço/Lucro "," {:,.2f}".format(Pl))

    with col15:
        st.metric(f"Dividend yield","  {:,.2f} %".format(dividend_yeld) )

In [12]:
with st.container():
    st.text("Preço das ações:")
    st.area_chart(df["Adj Close"])

    st.text("Mínimo, Preço das Ações e Máximo")
    st.line_chart(df[["Low","Adj Close", "High"]])

In [13]:
with st.container():
    st.subheader("Sobre os dividendos")
    st.caption(f"Nessa parte iremos analisar profundamente a questão dos Dividendos pagos pela {ativo}")

    col5, col6, col7 = st.columns(3)

    with col5:
        st.caption("Tabela com todos os pagamentos no intervalo: ")
        st.dataframe(data[["Dividends"]])

    with col6:
        st.metric(f"Quantidade de pagamentos no intervalo: ", " {:,.2f}".format(historico))
        st.metric(f"Pagamento médio (Dividendos): ", " R$ {:,.2f}".format(dividendo))

    with col7:
        st.metric(f"Pagamento máximo Dividendo", "R$ {:,.2f}".format(dividen_max))
        st.metric(f"Pagamento mínimo Dividendo", "R$ {:,.2f}".format(dividen_min))

In [14]:
with st.container():
    st.caption("Grafico de linhas com os pagamentos dos Dividendos:")
    st.line_chart(data["Dividends"])

In [15]:
with st.container():
    st.subheader("Simulador de lucros com Dividendo")
    number = st.number_input('Quanto seria o investimento?')
    st.write('O investimento é de: R$ ', number)
    lucro = number * (dividend_yeld/100)
    compra = number / ult_cotacao
    print(compra)
    col21, col22 = st.columns(2)
    with col21:

        st.caption("FÓMULA PARA CÁLCULO DE LUCRO: Dividend Yield x Valor Investido")
        st.metric(f"O lucro médio seria de: ", " R$ {:,.2f}".format(lucro))

    with col22:
        st.caption("Para ter esse lucro médio precisaria a quantidade de ações abaixo")
        st.metric(f"Equivalente a ", " {:,.0f} Ações".format(compra))

0.0
